In [1]:
import os

In [2]:
%pwd

'/Users/DEVASHAH/Desktop/Dev Coding/Chicken-Disease-Classification/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/DEVASHAH/Desktop/Dev Coding/Chicken-Disease-Classification'

## Update the entity

In [5]:
from dataclasses import dataclass
from pathlib import Path

## entity defined 
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [10]:
from CNNClassifier.constants import *
from CNNClassifier.utils.common import read_yaml, create_directories

## Update the Configuration Manager

In [11]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        ## this path read by read_yaml will give all 4  parameters that are written in config.yaml
        self.config_filepath = config_filepath  
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        ## this artifacts_root is the key which will give value artifact and directory will be created
        create_directories([self.config.artifacts_root])


    ## entity name here used as return type 
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        ## here we are creating data ingestion folder inside artifacts as written in config.yaml
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
      

## Data Ready to get Read

In [12]:
import os
import urllib.request as request
import zipfile
from CNNClassifier import logger
from CNNClassifier.utils.common import get_size

In [13]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)



## Update the Pipeline

In [14]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-11-12 10:33:28,758: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-11-12 10:33:28,760: INFO: common: yaml file: params.yaml loaded successfully]
[2024-11-12 10:33:28,761: INFO: common: created directory at: artifacts]
[2024-11-12 10:33:28,761: INFO: common: created directory at: artifacts/data_ingestion]
[2024-11-12 10:33:33,455: INFO: 1170291011: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 8123:3FA922:312A1:430C5:6732E19F
Accept-Ranges: bytes
Date: Tue, 12 Nov 2024 05:03:30 GMT
Via: 1.1 varnish
X-Served-By: cache-bom4721-BOM
X-Cache: MIS